In [106]:
import pandas as pd
import os
import numpy as np

INPUT_ASIM_CONFIGS = r'E:\Met_Council\metc-asim-model\source\activitysim\configs'
OUTPUT_ASIM_CONFIGS = r'E:\Met_Council\metc-asim-model\estimation\configs_wlc'
ALOGIT_OUTPUT = r'E:\Met_Council\metc-asim-model\estimation\output\work_loc_estimation\destination_choice\work_location_choice'

DC_SEGMENT = 'workplace' #model_selector

# (segment, [Fields]): 'alogit_variable'
# (segment, [Fields]): constant

SEG_IDS = ['work_low', 'work_med', 'work_high', 'work_veryhigh']

alogit2asim_size = {
    'ADJ_RET_EMP': ['sretlow', 'sretmed', 'srethi', 'sretvhi'],
    'ADJ_AMC_EMP': ['sindlow', 'sindmed', 'sindhi', 'sindvhi'],
    'ADJ_MAN_EMP': ['sindlow', 'sindmed', 'sindhi', 'sindvhi'],
    'ADJ_OFF_EMP': ['sofflow', 'soffmed', 'soffhi', 'soffvhi'],
    'ADJ_NRET_EMP': ['stotlow', 'stotmed', 'stothi', 'stotvhi']
}

alogit2workloc = {
    'coef_dist_zero_auto': 'bdzau',
    'coef_dist_student': 'bdstu',
    'coef_dist_part_time': 'bdemp',
    'coef_dist_vhigh_inc': 'bdvhi',
    'coef_dist_high_inc': 'bdhi',
    'coef_dist_med_inc': 'bdmed',
    'coef_dist_low_inc': 'bdlow',
    'coef_mode_logsum': 'blogsum', 
    'coef_dist_logged': 'bdistlg', 
    'coef_dist_cubed': 'bdistcu', 
    'coef_dist_squared': 'bdistsq', 
    'coef_dist': 'bdist',
    # 'coef_work_state': 'bstate',
    'coef_female': 'bdsex', 
    'coef_old': 'bdold', 
    'coef_young': 'bdyng',
    # 'coef_wi_to_mn': 'bwimn',
    # 'coef_mn_to_wi': 'bmnwi',
    'coef_ms_xing': 'bmsrx',
    'coef_mn_xing': 'bmnrx',
    'coef_sc_xing': 'bstcr'
}


# Read ALOGIT

In [107]:
colspec = [4, 10, 3, 21, 20]
colnames = ['x1', 'varname', 'fixed', 'estimate', 'std_err']
alogit_output = pd.read_fwf(os.path.join(ALOGIT_OUTPUT, 'Estimation.F12'), skiprows = 3, widths = colspec, names = colnames, na_values = '-')

if alogit_output['x1'].dtype == str:
    alogit_output = alogit_output[alogit_output['x1'].str.strip() == "0"]
else:
    alogit_output = alogit_output[alogit_output['x1'] == 0]
    

alogit_output['varname'] = alogit_output['varname'].str.strip()
alogit_output['estimate'] = alogit_output['estimate'].astype(float)
alogit_output['log_est'] = 0
alogit_output.loc[(alogit_output['varname'].str[0:1] == 's') & (alogit_output['estimate'] != 1.0), 'log_est'] = np.exp(alogit_output['estimate'])
alogit_output.loc[(alogit_output['varname'].str[0:1] == 's') & (alogit_output['estimate'] == 1.0), 'log_est'] = 1.0

alogit_output

,x1,varname,fixed,estimate,std_err,log_est
0,0.0,bdmx,F,-0.015771,.292095715430E-02,0.000000
1,0.0,bmsrx,F,-0.103542,.285430219854E-01,0.000000
2,0.0,bmnrx,F,-0.237010,.480276234965E-01,0.000000
3,0.0,bstcr,F,-0.312859,.100754635847,0.000000
4,0.0,bdzau,T,0.000000,.000000000000,0.000000
5,0.0,bdstu,F,-0.033174,.997244539033E-02,0.000000
6,0.0,bdemp,F,-0.039732,.436240870951E-02,0.000000
7,0.0,bmnwi,T,0.000000,.000000000000,0.000000
8,0.0,bwimn,T,0.000000,.000000000000,0.000000
9,0.0,bstate,T,0.000000,.000000000000,0.000000


# Read DC Size Terms

Then split, update, and output new to different folder

In [108]:
dc_size_terms = pd.read_csv(os.path.join(INPUT_ASIM_CONFIGS, 'destination_choice_size_terms.csv'), comment = "#")
output_dc_size_terms = dc_size_terms[dc_size_terms['model_selector'] != DC_SEGMENT].copy().set_index(['model_selector', 'segment'])
upd_dc_size_terms = dc_size_terms[dc_size_terms['model_selector'] == DC_SEGMENT].copy().set_index(['model_selector', 'segment'])
upd_dc_size_terms[upd_dc_size_terms.columns] = 0.0

for i, v in alogit2asim_size.items():
    for j, s in zip(v, SEG_IDS):
        if type(j) == float or type(j) == int:
            upd_dc_size_terms.loc[(DC_SEGMENT, s), i] = j
        else:
            if j in np.array(alogit_output['varname']):
                upd_var = alogit_output[alogit_output['varname'] == j][['estimate']].values[0][0]
                if upd_var != 1.0:
                    upd_dc_size_terms.loc[(DC_SEGMENT, s), i] = np.exp(upd_var)
                else:
                    upd_dc_size_terms.loc[(DC_SEGMENT, s), i] = 1.0


output_dc_size_terms = pd.concat([output_dc_size_terms, upd_dc_size_terms])
output_dc_size_terms.to_csv(os.path.join(OUTPUT_ASIM_CONFIGS, 'destination_choice_size_terms.csv'))

# Update coefficients file

In [109]:
work_loc_coef = pd.read_csv(os.path.join(INPUT_ASIM_CONFIGS, 'workplace_location_coefficients.csv')).set_index('coefficient_name')
for i, v in work_loc_coef.iterrows():
    if i in alogit2workloc and work_loc_coef.loc[i, 'constrain'] == 'F':
        upd_var = alogit_output[alogit_output['varname'] == alogit2workloc[i]][['estimate']].values[0][0]
        work_loc_coef.loc[i, 'value'] = upd_var
    else:
        if i in alogit2workloc:
            print(f"value {i} is constrained, not changing...")
        else:
            print(f"value {i} is not setup, no change made to it...")
    
work_loc_coef.to_csv(os.path.join(OUTPUT_ASIM_CONFIGS, 'workplace_location_coefficients.csv'))

value coef_work_state is not setup, no change made to it...
value coef_wi_to_mn is not setup, no change made to it...
value coef_mn_to_mn is not setup, no change made to it...
value coef_wi_to_wi is not setup, no change made to it...
value coef_mn_to_wi is not setup, no change made to it...
value coef_dest_mix is not setup, no change made to it...
